# Trabalho Final - Análise de Dados com Python

Integrantes:

    1) Camila Primo | RM 349380
    2) Caroline Claro | RM 350444
    3) Fernando Martins | RM 350532

### **Preparação do Ambiente**

In [5]:
import xgboost

In [6]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)
import xgboost; print("XGBoost", xgboost.__version__)

Linux-5.15.120+-x86_64-with-glibc2.35
Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
NumPy 1.23.5
SciPy 1.11.4
Scikit-Learn 1.2.2
XGBoost 2.0.2


In [7]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [9]:
from xgboost import XGBRegressor

In [10]:
from sklearn.metrics import mean_squared_error
import numpy as np

In [11]:
df_housing = pd.read_csv('housing.csv', sep=',')
df_housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [12]:
#transformando a coluna string em um dado categorico
df_housing = df_housing.join(pd.get_dummies(df_housing['ocean_proximity'], prefix='ocean_proximity', drop_first=True))
df_housing.drop('ocean_proximity', axis=1, inplace=True)

In [13]:
df_housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,1,0,0,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,1,0,0,0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,1,0,0,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,1,0,0,0


In [14]:
# Verificando quais colunas possuem valores faltantes (True = contém missings)
missings = df_housing.isna().any()
print(missings)

longitude                     False
latitude                      False
housing_median_age            False
total_rooms                   False
total_bedrooms                 True
population                    False
households                    False
median_income                 False
median_house_value            False
ocean_proximity_INLAND        False
ocean_proximity_ISLAND        False
ocean_proximity_NEAR BAY      False
ocean_proximity_NEAR OCEAN    False
dtype: bool


In [15]:
# Conta o total de missings e calcula quantos % essas observações representam do total do dataset
missings_count = df_housing['total_bedrooms'].isnull()

print('Total de observações com missing:', missings_count.sum())
print('Total de observações da base de dados:', len(df_housing))
print('Percentual da base com missings:', round((missings_count.sum()/len(df_housing)), 4))

Total de observações com missing: 207
Total de observações da base de dados: 20640
Percentual da base com missings: 0.01


In [16]:
# Como o total de observções com missing representa apenas cerca de 1% do dataset, optamos por não fazer o preenchimento e sim dropar essas observações
# O preenchimento geralmente leva a viés e a perda de amostra não é algo que compromete as estimativas

df_housing = df_housing.dropna()

In [17]:
# Dropando a categoria 'INLAND' (será a categoria base)
df_housing = df_housing.drop('ocean_proximity_INLAND', axis=1)

In [18]:
#separando os dados em features e labels
X = df_housing.iloc[:,:-1]
y = df_housing.iloc[:,-1]

In [19]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    df_housing.drop('median_house_value', axis=1), df_housing['median_house_value'], test_size=0.2, random_state=42
)

### **XGBoost**

In [ ]:
xg_reg = XGBRegressor()

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

In [ ]:
grid_search = GridSearchCV(estimator=xg_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}


In [ ]:
best_xg_reg = grid_search.best_estimator_
y_pred = best_xg_reg.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print("RMSE: %0.2f" % (rmse))

RMSE: 47363.79


In [ ]:
print((rmse))

47363.78964818601


### **LightGBM**

In [ ]:
import lightgbm as lgb

In [ ]:
lgb_reg = lgb.LGBMRegressor()

In [ ]:
param_grid_lgb = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


In [ ]:
grid_search_lgb = GridSearchCV(estimator=lgb_reg, param_grid=param_grid_lgb, scoring='neg_mean_squared_error', cv=5, verbose=1)

grid_search_lgb.fit(X_train, y_train)


A saída de streaming foi truncada nas últimas 5000 linhas.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001395 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1842
[LightGBM] [Info] Number of data points in the train set: 13076, number of used features: 10
[LightGBM] [Info] Start training from score 207551.346819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'colsample_bytree': [0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'subsample': [0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [ ]:
best_params_lgb = grid_search_lgb.best_params_
print("Best Parameters (LightGBM):", best_params_lgb)

Best Parameters (LightGBM): {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 0.8}


In [ ]:
best_lgb_reg = grid_search_lgb.best_estimator_
y_pred_lgb = best_lgb_reg.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [ ]:
mse_lgb = mean_squared_error(y_test, y_pred_lgb)
rmse_lgb = np.sqrt(mse_lgb)
print("RMSE (LightGBM): %0.2f" % (rmse_lgb))

RMSE (LightGBM): 47389.75


### **Random Forest**

In [20]:
from sklearn.ensemble import RandomForestRegressor

In [21]:
rf_reg = RandomForestRegressor()

In [22]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [23]:
grid_search_rf = GridSearchCV(estimator=rf_reg, param_grid=param_grid_rf, scoring='neg_mean_squared_error', cv=5, verbose=1)

In [24]:
grid_search_rf.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [3, 5, 7], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error', verbose=1)

In [28]:
best_params_rf = grid_search_rf.best_params_
print("Best Parameters (Random Forest):", best_params_rf)

Best Parameters (Random Forest): {'max_depth': 7, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}


In [30]:
best_rf_reg = grid_search_rf.best_estimator_
y_pred_rf = best_rf_reg.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
print("RMSE (Random Forest): %0.2f" % (rmse_rf))

RMSE (Random Forest): 63032.31


Conclusão

Os modelos de XGBoost e LightGBM tiveram o melhor resultado RMSE sobre a target, tendo como resultado os valores esperados abaixo de 55000, sendo 47363.79 e 47389.75 respectivamente. O Random Forest foi o que apresentou o pior resultado, sendo 63032.31 e o tempo de execução mais alta.